In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  22 17:16:55 2017
@author: saviokay
"""
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import time
import math
import dask.array as da
import graphviz

In [2]:
# Read Function For MODO3 & MODO6 Files.
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    print('Reading The Cloud Mask From MOD06_L2 Product:')
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    print('The Level-2 Cloud Mask Array Shape',CM.shape)
    print(' ')

    myd03 = Dataset(MOD03_file, "r")
    print('Reading The Latitude-Longitude From MOD03 Product:')
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    print('The Level-2 Latitude-Longitude Array Shape',latitude.shape)
    print(' ')

    return latitude,longitude,CM

In [3]:
# Misc Function For Processing Cloud Fraction.
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')

    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None 

    return div

def countzero(x, axis=1):
    #print(x)
    count0 = 0
    count1 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    #print(count0/len(x))
    return count0/len(x)

In [4]:
# Setting File Location As Environment Variables
%env MOD03_PATH=/home/jovyan/Files/
%env MOD06_PATH=/home/jovyan/Files/

env: MOD03_PATH=/home/jovyan/Files/
env: MOD06_PATH=/home/jovyan/Files/


In [5]:
MOD03_path = %env MOD03_PATH
MOD06_path = %env MOD06_PATH
satellite = 'Aqua'

yr = [2008]
mn = [1] #np.arange(1,13)  #[1]
dy = [1] #np.arange(1,32) # [1] #np.arange(1,31)
lat_bnd = np.arange(-90,91,1)# latitude and longtitude boundaries of level-3 grid
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

TOT_pix      = np.zeros(nlat*nlon)
CLD_pix      = np.zeros(nlat*nlon)

In [6]:
MOD03_fp = 'MYD03.A*.hdf'
MOD06_fp = 'MYD06_L2.A*.hdf'
MOD03_fn, MOD06_fn =[],[]
#MOD03_fn2, MOD06_fn2 =[],[]
MOD03_fn2, MOD06_fn2 =[],[]
for MOD06_flist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_flist, MOD06_fp):
        MOD06_fn = MOD06_flist
        MOD06_fn2.append(MOD06_flist)
        #print(MOD06_fn)
for MOD03_flist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_flist, MOD03_fp):
        MOD03_fn = MOD03_flist
        MOD03_fn2.append(MOD03_flist)
        #print(MOD03_fn)
if MOD03_fn and MOD06_fn:
    # if both MOD06 and MOD03 products are in the directory
    print('Reading Level 2 GeoLocation & Cloud Data')
    #print(MOD06_fn)
    #print(MOD03_fn)
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_fn,MOD03_path+MOD03_fn)
    

Reading Level 2 GeoLocation & Cloud Data
Reading The Cloud Mask From MOD06_L2 Product:
The Level-2 Cloud Mask Array Shape (2030, 1354)
 
Reading The Latitude-Longitude From MOD03 Product:
The Level-2 Latitude-Longitude Array Shape (2030, 1354)
 


In [7]:
print('The Files In The MODO3 List: ')
print(MOD03_fn2)
print(' ')
print('The Files In The MODO6_L2 List: ')
print(MOD06_fn2)

The Files In The MODO3 List: 
['MYD03.A2008001.0010.006.2012066122416.hdf', 'MYD03.A2008001.0000.006.2012066122450.hdf', 'MYD03.A2008001.0005.006.2012066122516.hdf']
 
The Files In The MODO6_L2 List: 
['MYD06_L2.A2008001.0005.006.2013341193207.hdf', 'MYD06_L2.A2008001.0010.006.2013341192125.hdf', 'MYD06_L2.A2008001.0000.006.2013341193524.hdf']


In [8]:
myd06_name = '/home/jovyan/Files/'

cm = np.zeros((2030,1354), dtype=np.float32)
bit0 = np.zeros((2030,1354), dtype=np.float32)
bit12 = np.zeros((2030,1354), dtype=np.float32)

for MOD06_file in MOD06_fn2:
    MOD06_file2 = myd06_name + MOD06_file
    myd06 = Dataset(MOD06_file2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]
    bit0r   = (np.array(CM,dtype='byte') & 0b00000001)
    bit12r   = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    cm = np.dstack((cm,CM))
    bit0 = np.dstack((bit0,bit0r))
    bit12 = np.dstack((bit12,bit12r))
        
print('The Cloud Mask Array Shape Is: ',cm.shape)

myd03_name = '/home/jovyan/Files/'


lat = np.zeros((2030,1354), dtype=np.float32)
lon = np.zeros((2030,1354), dtype=np.float32)

for MOD03_file in MOD03_fn2:
    MOD03_file2 = myd03_name + MOD03_file
    myd03 = Dataset(MOD03_file2, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    lat = np.dstack((lat,latitude))

    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    lon = np.dstack((lon,longitude))


    
print('The Latitude Array Shape Is: ',lat.shape)
print('The Longitude Array Shape Is: ',lon.shape)
    

The Cloud Mask Array Shape Is:  (2030, 1354, 4)
The Latitude Array Shape Is:  (2030, 1354, 4)
The Longitude Array Shape Is:  (2030, 1354, 4)


In [9]:
latint = lat.astype(np.int8)
lonint = lon.astype(np.int8)

In [10]:
lat = da.from_array(lat, chunks =(2030,1354,1))
lon = da.from_array(lon, chunks =(2030,1354,1))
latint = da.from_array(latint, chunks =(2030,1354,1))
lonint = da.from_array(lonint, chunks =(2030,1354,1))
cm = da.from_array(cm, chunks =(2030,1354,1))
bit0 = da.from_array(bit0, chunks =(2030,1354,1))
bit12 = da.from_array(bit12, chunks =(2030,1354,1))

In [11]:
dsa = xr.Dataset()
dsa.coords['Latitude'] = (('x','y','z'), lat)
dsa.coords['Longitude'] = (('x','y','z'), lon)
dsa.coords['LatInt'] = (('x','y','z'), latint)
dsa.coords['LonInt'] = (('x','y','z'), lonint)
dsa['CloudMask'] = (('x','y','z'), cm)

In [12]:
from dask.distributed import Client

client = Client("tcp://10.49.141.6:41239")
client

Client Scheduler: tcp://10.49.141.6:41239 Dashboard: http://10.49.141.6:8787/status,Cluster Workers: 6 Cores: 30 Memory: 206.02 GB


In [ ]:
start_time = time.time()

listOfCM = []
finallist = []

dsag = list(dsa.groupby('LonInt'))
[future] = client.scatter([dsa], broadcast=True)  
for listOfOneLong in dsag:
    oneGrid = listOfOneLong[1].groupby('LatInt').reduce(countzero)
    for values in oneGrid:
        finalvalues = [oneGrid.LatInt.data[0], listOfOneLong[0], values.CloudMask.data[0]]
        listOfCM.append(values)
        finallist.append(finalvalues)
    
    #print(oneGrid)
    #for one in list(oneGrid.Latitude):
       # print(listOfOneLong[0], one, oneGrid.CM[1])

        
end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [16]:
start_time = time.time()

listOfCM = []
finallist = []

dsag = list(dsa.groupby('LonInt'))

for listOfOneLong in dsag:
    #oneGrid = listOfOneLong[1].groupby('LatInt').reduce(countzero)
    big_future = client.scatter(listOfOneLong[1].groupby('LatInt'))
    future = client.submit(xr.Dataset.reduce(countzero), big_future)  
    for values in oneGrid:
        finalvalues = [oneGrid.LatInt.data[0], listOfOneLong[0], values.CloudMask.data[0]]
        listOfCM.append(values)
        finallist.append(finalvalues)
    
    #print(oneGrid)
    #for one in list(oneGrid.Latitude):
       # print(listOfOneLong[0], one, oneGrid.CM[1])

        
end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

TypeError: reduce() missing 1 required positional argument: 'func'

In [ ]:
start_time = time.time()

#listOfCM = []
#finallist = []

dsag = list(dsa.groupby('LonInt'))
[future] = client.scatter([dsa], broadcast=True)  
for listOfOneLong in dsag:
    oneGrid = listOfOneLong[1].groupby('LatInt').reduce(countzero)
    for values in oneGrid:
        print(values)
        #finalvalues = [oneGrid.LatInt.data[0], listOfOneLong[0], values[0]]
        #listOfCM.append(values)
        #finallist.append(finalvalues)
    
    #print(oneGrid)
    #for one in list(oneGrid.Latitude):
       # print(listOfOneLong[0], one, oneGrid.CM[1])

        
end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [32]:
for values in oneGrid:
    print(values)

CloudMask


In [30]:
len(finallist)

57

In [31]:
finallist

[[0, 0, 'C'],
 [16, 11, 'C'],
 [16, 12, 'C'],
 [15, 13, 'C'],
 [15, 14, 'C'],
 [15, 15, 'C'],
 [15, 16, 'C'],
 [15, 17, 'C'],
 [15, 18, 'C'],
 [15, 19, 'C'],
 [15, 20, 'C'],
 [14, 21, 'C'],
 [14, 22, 'C'],
 [14, 23, 'C'],
 [14, 24, 'C'],
 [14, 25, 'C'],
 [14, 26, 'C'],
 [14, 27, 'C'],
 [13, 28, 'C'],
 [13, 29, 'C'],
 [13, 30, 'C'],
 [13, 31, 'C'],
 [13, 32, 'C'],
 [14, 33, 'C'],
 [17, 34, 'C'],
 [21, 35, 'C'],
 [24, 36, 'C'],
 [26, 37, 'C'],
 [29, 38, 'C'],
 [31, 39, 'C'],
 [34, 40, 'C'],
 [36, 41, 'C'],
 [38, 42, 'C'],
 [40, 43, 'C'],
 [41, 44, 'C'],
 [43, 45, 'C'],
 [44, 46, 'C'],
 [46, 47, 'C'],
 [47, 48, 'C'],
 [49, 49, 'C'],
 [50, 50, 'C'],
 [51, 51, 'C'],
 [52, 52, 'C'],
 [53, 53, 'C'],
 [54, 54, 'C'],
 [55, 55, 'C'],
 [56, 56, 'C'],
 [56, 57, 'C'],
 [57, 58, 'C'],
 [58, 59, 'C'],
 [59, 60, 'C'],
 [59, 61, 'C'],
 [60, 62, 'C'],
 [60, 63, 'C'],
 [61, 64, 'C'],
 [61, 65, 'C'],
 [62, 66, 'C']]

In [24]:
start_time = time.time()

listOfCM = []
finallist = []

dsag = list(dsa.groupby('LonInt'))
[future] = client.scatter([dsa], broadcast=True)  
for listOfOneLong in dsag:
    oneGrid = listOfOneLong[1].groupby('LatInt').reduce(countzero)
    for values in oneGrid:
        finalvalues = [oneGrid.LatInt.data[0], listOfOneLong[0], oneGrid.CloudMask.data[0]]
        listOfCM.append(values)
    finallist.append(finalvalues)   
    
    #print(oneGrid)
    #for one in list(oneGrid.Latitude):
       # print(listOfOneLong[0], one, oneGrid.CM[1])



end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Total Time Taken This Loop:  481.0655539035797
00:08:01.07


In [25]:
finallist

[[0, 0, 1.0],
 [16, 11, 0.0],
 [16, 12, 0.0],
 [15, 13, 0.0],
 [15, 14, 0.0],
 [15, 15, 0.0],
 [15, 16, 0.0],
 [15, 17, 0.0],
 [15, 18, 0.0],
 [15, 19, 0.0],
 [15, 20, 0.0],
 [14, 21, 0.0],
 [14, 22, 0.0],
 [14, 23, 0.0],
 [14, 24, 0.0],
 [14, 25, 0.0],
 [14, 26, 0.0],
 [14, 27, 0.0],
 [13, 28, 0.0],
 [13, 29, 0.6880146386093321],
 [13, 30, 0.9236111111111112],
 [13, 31, 1.0],
 [13, 32, 1.0],
 [14, 33, 1.0],
 [17, 34, 0.7948717948717948],
 [21, 35, 0.8691588785046729],
 [24, 36, 0.941544885177453],
 [26, 37, 1.0],
 [29, 38, 1.0],
 [31, 39, 1.0],
 [34, 40, 0.8995098039215687],
 [36, 41, 0.9746835443037974],
 [38, 42, 0.8850174216027874],
 [40, 43, 1.0],
 [41, 44, 1.0],
 [43, 45, 0.9712918660287081],
 [44, 46, 1.0],
 [46, 47, 1.0],
 [47, 48, 1.0],
 [49, 49, 0.9104084321475626],
 [50, 50, 0.7057692307692308],
 [51, 51, 0.48205128205128206],
 [52, 52, 1.0],
 [53, 53, 1.0],
 [54, 54, 1.0],
 [55, 55, 1.0],
 [56, 56, 0.28554502369668244],
 [56, 57, 0.0],
 [57, 58, 0.0],
 [58, 59, 0.0981963927

In [26]:
len(finallist)

57

In [27]:
print([oneGrid.LatInt.data[0], listOfOneLong[0], oneGrid.CloudMask.data[0]])

[62, 66, 1.0]


In [28]:
listOfOneLong[1].LatInt.compute()

<xarray.DataArray 'LatInt' (stacked_x_y_z: 44)>
array([62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
       62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
       62, 62, 62, 62, 62, 62, 62, 62], dtype=int8)
Coordinates:
    Latitude       (stacked_x_y_z) float32 62.6756 62.64741 ... 62.41523
    Longitude      (stacked_x_y_z) float32 66.012054 66.081024 ... 66.00171
    LatInt         (stacked_x_y_z) int8 62 62 62 62 62 62 ... 62 62 62 62 62 62
    LonInt         (stacked_x_y_z) int8 66 66 66 66 66 66 ... 66 66 66 66 66 66
  * stacked_x_y_z  (stacked_x_y_z) MultiIndex
  - x              (stacked_x_y_z) int64 0 0 0 0 0 1 1 1 1 ... 5 5 6 6 7 7 8 8 9
  - y              (stacked_x_y_z) int64 1349 1350 1351 1352 ... 1352 1353 1353
  - z              (stacked_x_y_z) int64 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2

In [ ]:
listOfOneLong[1].LonInt.compute()